# Format and Update All DAX Measures in Semantic Model

This script connects to a Microsoft Fabric semantic model, iterates through all tables and their DAX measures, formats each measure’s DAX expression using the online service [daxformatter.com](https://www.daxformatter.com), and updates the model with the formatted expressions. It includes error checking to skip measures that fail to format, ensuring only valid and improved DAX is written back to the model. This helps maintain clean, readable, and consistent DAX code across your semantic datasets.


Installing semantic link labs

In [ ]:
%pip install semantic-link-labs

Importing Modules

In [ ]:
import requests
from bs4 import BeautifulSoup
from sempy_labs.tom import connect_semantic_model

Defining a Method in Python

In [ ]:
def format_and_update_all_measures(
    workspace: str,
    dataset: str,
    region: str = 'US',
    line: str = 'short'
) -> None:
    """
    Connects to a Microsoft Fabric semantic model and updates all DAX measures
    with formatted DAX expressions retrieved from daxformatter.com. Includes error checking
    to skip measures that fail formatting or are already formatted.

    This function helps maintain clean, readable, and consistent DAX code
    across all measures in the specified semantic dataset.

    Parameters
    ----------
    workspace : str
        The Fabric workspace name or ID containing the semantic model.
    dataset : str
        The semantic dataset name within the workspace.
    region : str, default 'US'
        Region code used by daxformatter.com for formatting preferences.
    line : str, default 'short'
        Formatting style ('short' or other) used by daxformatter.com.

    Returns
    -------
    None
        This function updates measures in place and prints status messages
        but does not return a value.
    """
    def format_dax(dax_expression: str) -> str | None:
        """
        Sends a DAX expression to daxformatter.com and returns the formatted DAX string.
        Returns None if formatting fails.

        Parameters
        ----------
        dax_expression : str
            The DAX expression to format.

        Returns
        -------
        str or None
            The formatted DAX expression, or None if formatting failed.
        """
        url = "https://www.daxformatter.com"
        payload = {
            'r': region,
            'fx': dax_expression,
            'embed': '1',
            's': 'auto',
            'l': line
        }
        try:
            response = requests.post(url, data=payload, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            formatted_div = soup.find('div', {'class': 'formatted'})
            if not formatted_div:
                return None
            for br in formatted_div.find_all("br"):
                br.replace_with("\n")
            text = formatted_div.get_text(' ').replace('\xa0', ' ')
            return text.strip()
        except Exception as e:
            print(f"[ERROR] Failed to format DAX: {e}")
            return None

    with connect_semantic_model(workspace=workspace, dataset=dataset, readonly=False) as tom:
        for table in tom.model.Tables:
            for measure in table.Measures:
                original_expr = measure.Expression
                formatted_expr = format_dax(original_expr)

                if formatted_expr is None:
                    print(f"[SKIPPED] Measure '{measure.Name}' in table '{table.Name}' could not be formatted.")
                    continue

                if formatted_expr != original_expr:
                    try:
                        tom.update_measure(
                            measure_name=measure.Name,
                            expression=formatted_expr
                        )
                        print(f"[UPDATED] {measure.Name}")
                    except Exception as e:
                        print(f"[ERROR] Failed to update measure '{measure.Name}': {e}")
                else:
                    print(f"[OK] Measure '{measure.Name}' is already formatted.")


Example Calling the Notebook

In [ ]:
format_and_update_all_measures(
    workspace="Dev",
    dataset="Dynamic Highlighting in Power BI with Visual Calcs"
)